***GENERATED CODE FOR testmanual PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def calculateFormula(df, functionsData, applyOn):
    dfcp = df
    for functionData in functionsData:
        if functionData['useNext']:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
            else:
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['operand1'], functionData['operand2'])
        else:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData['operator'],
                                                            functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                dfcp.drop('dataPrepColumn_Inprocess', axis=1)
            else:
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
    return dfcp


def calculateEquation(df, operator, columnA, columnB):
    if (operator == "add"):
        return df[columnA].add(df[columnB])
    elif (operator == "sub"):
        return df[columnA].sub(df[columnB])
    elif (operator == "mul"):
        return df[columnA].mul(df[columnB])
    elif (operator == "div"):
        return df[columnA].div(df[columnB])
    elif (operator == "mod"):
        return df[columnA].mod(df[columnB])


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '00AA43CC-6CAB-4E2F-900B-A4A3241FCBD5', 'max': 'FFE66FB5-8395-480F-8AE5-69A78EEAF750', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfEntry'}, {'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfService'}, {'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '90912.91', 'stddev': '6487.02', 'min': '67028', 'max': 'J9035', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B39.4', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'C69.21', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F7A05103-C8F5-4B0E-9D5D-6431124B8F84', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature_label': 'PayerId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerId')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pyspark


def linear_regression(sparkDF, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    lr_model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    lr_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % lr_model.score(X_test, y_test))

    data = {'model': lr_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run testmanualHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\t\t\t\tSub2.PayerId\r\nFROM \r\n(\r\n\t\tSELECT \r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\t\tpp.ProviderID,\r\n\t\tcptc.CPTCode,\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\td.DepartmentCategory,\r\n\t\ts.SpecialityCategory,\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\t\tfrom Transactions t\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\t\tand t.DateOfEntry between '1/1/2020' and '10/30/2020'  \r\n) as sub2\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\t\t\t t.DateOfEntry,\r\n\t\t\t count(*) as total_transactions\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tWHERE t.Amount > 0.1 \r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n) as sub1\r\n on sub1.cid = sub2.ChargeDetailID\r\n where sub2.Gender is not NULL\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\t\t\t\tSub2.PayerId", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run testmanualHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(sqlserver,spark,json.dumps( {"url": null, "source_attributes": {"host": "wsc-sqlserver1.database.windows.net", "port": "1433", "password": "G5gTW5rnKVeAZWXm", "database": "Demodb2020", "optionalDB": "Demodb2020", "user": "Numtra", "qtype": "query", "query": "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\t\t\t\tSub2.PayerId\r\nFROM \r\n(\r\n\t\tSELECT \r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\t\tpp.ProviderID,\r\n\t\tcptc.CPTCode,\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\td.DepartmentCategory,\r\n\t\ts.SpecialityCategory,\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\t\tfrom Transactions t\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\t\tand t.DateOfEntry between '1/1/2020' and '10/30/2020'  \r\n) as sub2\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\t\t\t t.DateOfEntry,\r\n\t\t\t count(*) as total_transactions\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tWHERE t.Amount > 0.1 \r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n) as sub1\r\n on sub1.cid = sub2.ChargeDetailID\r\n where sub2.Gender is not NULL\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\t\t\t\tSub2.PayerId", "dbtype": "mssql", "is_header": "Use Header Line", "server_url": "/numtraPlatform/NumtraPlatformV2/uploads/platform/"}, "DataPrepFile": null, "data_source": "mssql", "startListenerOnly": 1, "FilePath": "/FileStore/platform/extra/6066b94408f39555c41a8c821617345255/0part.csv", "requestRatio": 296.0, "totalRows": 147934, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 14, "numberOfRows": 147934, "duplicateRowCount": 0, "stats": [{"column": "ChargeDetailID", "alias": "ChargeDetailID", "type": "String", "max": "FFFF91CB-4D2B-48E7-AAE0-D2CB35E5B8E3", "min": "0000F12A-424D-43D9-A046-BA4208D30CBD", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Age", "alias": "Age", "type": "real", "max": 108.0, "min": 2.0, "mean": 71.4783822515446, "missing": 0.0, "stddev": 14.96}, {"column": "Gender", "alias": "Gender", "type": "String", "max": "M", "min": "F", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DateOfEntry", "alias": "DateOfEntry", "type": "String", "max": "2020-10-30T00:00:00.000Z", "min": "2020-01-02T00:00:00.000Z", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DateOfService", "alias": "DateOfService", "type": "String", "max": "2020-10-30T00:00:00.000Z", "min": "2017-08-10T00:00:00.000Z", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "ProviderID", "alias": "ProviderID", "type": "String", "max": "CBBD4925-1757-45BC-A853-1C2CA203B9E3", "min": "03151B5D-685B-4211-B01F-7BBB1C4A8A49", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "CPTCode", "alias": "CPTCode", "type": "String", "max": "S9981", "min": "65222", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "pdiagnosis1", "alias": "pdiagnosis1", "type": "String", "max": "Z98.890", "min": "A51.43", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "pdiagnosis2", "alias": "pdiagnosis2", "type": "String", "max": "Z98.890", "min": "A15.0", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DepartmentCategory", "alias": "DepartmentCategory", "type": "String", "max": "Pulmonary", "min": "Cardiology", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "SpecialityCategory", "alias": "SpecialityCategory", "type": "String", "max": "Ophtamology", "min": "Ophtamology", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "days_to_first_post", "alias": "days_to_first_post", "type": "real", "max": 355.0, "min": -667.0, "mean": 27.74492679167737, "missing": 0.0, "stddev": 73.35}, {"column": "PayerId", "alias": "PayerId", "type": "String", "max": "FF58D17E-F6AC-4390-999A-AE5378E83883", "min": "012FD8A9-37BE-4D13-B808-7E6742223444", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "New_Age_add_days_to_first_post", "alias": "New_Age_add_days_to_first_post", "type": "real", "max": 442.0, "min": -627.0, "mean": 99.22330904322197, "missing": 0.0, "stddev": 76.28}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "formula", "applyOn": [{"columnName": "Age", "type": "real", "min": 2, "max": 108, "mean": 71.5}, {"columnName": "days_to_first_post", "type": "real", "min": -667, "max": 355, "mean": 27.7}], "functionsData": [{"operand1": "Age", "operator": "add", "operand2": "days_to_first_post", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "Age", "type": "real", "min": 2, "max": 108, "mean": 71.5}}, {"Columndata": {"columnName": "days_to_first_post", "type": "real", "min": -667, "max": 355, "mean": 27.7}}, {"Columndata": ""}]}]}], "functionChanges": [{"columnName": "Age", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "Age", "operator": "add", "operand2": "days_to_first_post", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "Age", "type": "real", "min": 2, "max": 108, "mean": 71.5}}, {"Columndata": {"columnName": "days_to_first_post", "type": "real", "min": -667, "max": 355, "mean": 27.7}}, {"Columndata": ""}]}]}, {"columnName": "days_to_first_post", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "Age", "operator": "add", "operand2": "days_to_first_post", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "Age", "type": "real", "min": 2, "max": 108, "mean": 71.5}}, {"Columndata": {"columnName": "days_to_first_post", "type": "real", "min": -667, "max": 355, "mean": 27.7}}, {"Columndata": ""}]}]}], "fileheader": [{"field": "ChargeDetailID", "alias": "ChargeDetailID", "generated": 0, "position": 1, "type": "String"}, {"field": "Age", "alias": "Age", "generated": 0, "position": 2, "type": "real"}, {"field": "Gender", "alias": "Gender", "generated": 0, "position": 3, "type": "String"}, {"field": "DateOfEntry", "alias": "DateOfEntry", "generated": 0, "position": 4, "type": "String"}, {"field": "DateOfService", "alias": "DateOfService", "generated": 0, "position": 5, "type": "String"}, {"field": "ProviderID", "alias": "ProviderID", "generated": 0, "position": 6, "type": "String"}, {"field": "CPTCode", "alias": "CPTCode", "generated": 0, "position": 7, "type": "String"}, {"field": "pdiagnosis1", "alias": "pdiagnosis1", "generated": 0, "position": 8, "type": "String"}, {"field": "pdiagnosis2", "alias": "pdiagnosis2", "generated": 0, "position": 9, "type": "String"}, {"field": "DepartmentCategory", "alias": "DepartmentCategory", "generated": 0, "position": 10, "type": "String"}, {"field": "SpecialityCategory", "alias": "SpecialityCategory", "generated": 0, "position": 11, "type": "String"}, {"field": "days_to_first_post", "alias": "days_to_first_post", "generated": 0, "position": 12, "type": "real"}, {"field": "PayerId", "alias": "PayerId", "generated": 0, "position": 13, "type": "String"}, {"field": "New_Age_add_days_to_first_post", "alias": "New_Age_add_days_to_first_post", "generated": 0, "position": 14, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run testmanualHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "00AA43CC-6CAB-4E2F-900B-A4A3241FCBD5", "max": "FFE66FB5-8395-480F-8AE5-69A78EEAF750", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "67.18", "stddev": "15.9", "min": "4.0", "max": "101.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "DateOfEntry", "transformation_label": "Extract Date"}], "feature": "DateOfEntry", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfEntry"}, {"transformationsData": [{"feature_label": "DateOfService", "transformation_label": "Extract Date"}], "feature": "DateOfService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfService"}, {"transformationsData": [{"feature_label": "ProviderID", "transformation_label": "String Indexer"}], "feature": "ProviderID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "03151B5D-685B-4211-B01F-7BBB1C4A8A49", "max": "CBBD4925-1757-45BC-A853-1C2CA203B9E3", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderID"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "90912.91", "stddev": "6487.02", "min": "67028", "max": "J9035", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B39.4", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "C69.21", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Cardiology", "max": "Pulmonary", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Ophtamology", "max": "Ophtamology", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_to_first_post", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "15.61", "stddev": "58.76", "min": "-543.0", "max": "201.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "days_to_first_post"}, {"transformationsData": [{"feature_label": "PayerId", "transformation_label": "String Indexer"}], "feature": "PayerId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "03E92275-9F7C-4D34-8701-CE4C72CDE9F7", "max": "F7A05103-C8F5-4B0E-9D5D-6431124B8F84", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "New_Age_add_days_to_first_post", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "80.19", "stddev": "74.83", "min": "-555.0", "max": "314.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "New_Age_add_days_to_first..."}, {"feature": "ChargeDetailID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "249.5", "stddev": "144.48", "min": "0.0", "max": "499.0", "missing": "0"}, "updatedLabel": "ChargeDetailID_stringinde..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.45", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DateOfEntry_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "16.5", "stddev": "8.68", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "DateOfEntry_dayofmonth"}, {"feature": "DateOfEntry_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "5.63", "stddev": "2.79", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "DateOfEntry_month"}, {"feature": "DateOfEntry_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "DateOfEntry_year"}, {"feature": "DateOfService_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.89", "stddev": "8.52", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "DateOfService_dayofmonth"}, {"feature": "DateOfService_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "5.58", "stddev": "2.78", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "DateOfService_month"}, {"feature": "DateOfService_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "DateOfService_year"}, {"feature": "ProviderID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "5.31", "stddev": "3.31", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "ProviderID_stringindexer_..."}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "2.63", "stddev": "4.34", "min": "0.0", "max": "25.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "33.67", "stddev": "37.61", "min": "0.0", "max": "146.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "44.39", "stddev": "51.45", "min": "0.0", "max": "140.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.02", "stddev": "1.35", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PayerId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "4.93", "stddev": "8.78", "min": "0.0", "max": "47.0", "missing": "0"}, "updatedLabel": "PayerId_stringindexer_tra..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run testmanualHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=linear_regression(autofe, ["ChargeDetailID_stringindexer", "Age", "Gender_stringindexer", "ProviderID_stringindexer", "CPTCode_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PayerId_stringindexer", "New_Age_add_days_to_first_post", "DateOfEntry_dayofmonth", "DateOfEntry_month", "DateOfEntry_year", "DateOfService_dayofmonth", "DateOfService_month", "DateOfService_year"], "days_to_first_post")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
